In [47]:
from decouple import config

In [48]:
assert config("POLYGON_API_KEY", default=None, cast=str) is not None

In [49]:
import requests

In [50]:
import pytz
from dataclasses import dataclass
from datetime import datetime
from typing import Literal
from urllib.parse import urlencode

POLYGON_API_KEY = config("POLYGON_API_KEY", default=None, cast=str)

def transform_polygon_result(result):
    unix_timestamp = result.get('t') / 1000
    utc_timestamp = datetime.fromtimestamp(unix_timestamp, tz=pytz.timezone('UTC'))
    return {
     'open_price': result['o'],
     'close_price': result['c'],
     'high_price': result['h'],
     'low_price': result['l'],
     'number_of_trades': result['n'],
     'volume': result['v'],
     'volume_weighted_average': result['vw'],
     'time': utc_timestamp,
    }

@dataclass
class PolygonAPIClient:
    ticker: str = "AAPL"
    multiplier: int = "5"
    timespan: str = "minute" 
    from_date: str = "2023-01-09"
    to_date: str = "2023-01-09"
    api_key: str = ""
    adjusted: bool = True
    sort: Literal["asc", "desc"] = "asc"

    def get_api_key(self):
        return self.api_key or POLYGON_API_KEY

    # don't want to expose API key, so pass in headers instead
    def get_headers(self):
        api_key = self.get_api_key()
        return {
            "Authorization": f"Bearer {api_key}"
        }

    def get_params(self):
        return {
            "adjusted" : self.adjusted,
            "sort" : self.sort,
        }

    def generate_url(self, pass_auth=False):
        path = f"/v2/aggs/ticker/{self.ticker}/range/{self.multiplier}/{self.timespan}/{self.from_date}/{self.to_date}"
        url = f"https://api.polygon.io{path}"
        params = self.get_params()
        encoded_params = urlencode(params)
        url = f"{url}?{encoded_params}" # appends base url with encoded params
        if pass_auth:
            api_key = self.get_api_key()
            url += "&api_key={api_key}"
        return url

    # fetching the raw data
    def perform_request(self):
        headers = self.get_headers()
        url = self.generate_url()
        response = requests.get(url, headers=headers)
        response.raise_for_status() # not successful -> raises errors
        return response.json()

    
    def get_stock_data(self):
        data = self.perform_request() # making GET request to Polygon API
        results = data['results'] # accesses results key in data dictionary 
        dataset = []
        for result in results: # calls function on each results and stores in dataset
            dataset.append(
                transform_polygon_result(result)
            )
        return dataset

In [51]:
stock_api_client = PolygonAPIClient(ticker="GOOG", multiplier=1)
stock_api_client.generate_url()

'https://api.polygon.io/v2/aggs/ticker/GOOG/range/1/minute/2023-01-09/2023-01-09?adjusted=True&sort=asc'

In [52]:
dataset = stock_api_client.perform_request()
dataset

{'ticker': 'GOOG',
 'queryCount': 568,
 'resultsCount': 568,
 'adjusted': True,
 'results': [{'v': 1684,
   'vw': 88.1217,
   'o': 88.08,
   'c': 88.16,
   'h': 88.17,
   'l': 88.08,
   't': 1673254800000,
   'n': 55},
  {'v': 360,
   'vw': 88.3648,
   'o': 88.37,
   'c': 88.37,
   'h': 88.37,
   'l': 88.37,
   't': 1673254920000,
   'n': 12},
  {'v': 1209,
   'vw': 88.5577,
   'o': 88.52,
   'c': 88.55,
   'h': 88.55,
   'l': 88.52,
   't': 1673255100000,
   'n': 32},
  {'v': 2169,
   'vw': 88.548,
   'o': 88.55,
   'c': 88.55,
   'h': 88.55,
   'l': 88.55,
   't': 1673255160000,
   'n': 14},
  {'v': 152,
   'vw': 88.4496,
   'o': 88.42,
   'c': 88.42,
   'h': 88.42,
   'l': 88.42,
   't': 1673255220000,
   'n': 5},
  {'v': 168,
   'vw': 88.3512,
   'o': 88.35,
   'c': 88.35,
   'h': 88.35,
   'l': 88.35,
   't': 1673255280000,
   'n': 5},
  {'v': 219,
   'vw': 88.3498,
   'o': 88.35,
   'c': 88.35,
   'h': 88.35,
   'l': 88.35,
   't': 1673255400000,
   'n': 4},
  {'v': 814,
   'vw':